# Multi-Agents Orchestration Notebook

Ce notebook illustre un pipeline générique pour **finaliser un notebook cible** (quel qu’il soit), via la collaboration d’agents automatiques :
- **CoderAgent** : capable d’analyser et modifier le code ou le texte de cellules.
- **ReviewerAgent** : exécute le notebook, repère les problèmes potentiels et demande des corrections.
- **AdminAgent** : valide ou non la version finale.

**Utilisation** :
1. Sélectionner un notebook à finaliser (ou indiquer son chemin).
2. (Optionnel) Indiquer un objectif général (dans un champ texte).
3. Lancer la boucle d’orchestration et observer la conversation multi-agents.
4. Une fois le notebook approuvé par l’Admin, la conversation s’arrête automatiquement.

---


### Configuration de l'environnement

Cette cellule installe les dépendances nécessaires au bon fonctionnement du notebook :
- `papermill` pour l'exécution paramétrable de notebooks
- `nbformat` pour la manipulation des fichiers Jupyter
- `ipywidgets` pour les interactions utilisateur
- `semantic-kernel` pour la gestion des agents conversationnels

In [36]:
# Cellule Code : Installation
%pip install papermill nbformat ipywidgets semantic-kernel --quiet


Note: you may need to restart the kernel to use updated packages.


### Import des bibliothèques

Importation des composants essentiels :
- Gestion de fichiers et JSON
- Logging avec personnalisation des couleurs
- Manipulation de notebooks via `nbformat`
- Intégration du Semantic Kernel pour les agents IA

In [37]:
# Cellule Code : Imports et configuration
import os
import json
import hashlib
import logging
import nbformat
import papermill as pm
from datetime import datetime

# Pour la conversation multi-agents
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# On importe la base pour nos stratégies
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent, AuthorRole

### Journalisation avancée

Configuration d'un système de logging personnalisé :
- Niveaux de log colorés (DEBUG/INFO/WARNING/ERROR)
- Formatage horodaté des messages
- Sortie console avec gestion des verbosités

In [38]:
class ColorFormatter(logging.Formatter):
    COLORS = {
        'DEBUG': '\033[94m',    # Blue
        'INFO': '\033[92m',     # Green
        'WARNING': '\033[93m',  # Yellow
        'ERROR': '\033[91m',    # Red
        'CRITICAL': '\033[91m'  # Red
    }
    RESET = '\033[0m'

    def format(self, record):
        color = self.COLORS.get(record.levelname, '')
        message = super().format(record)
        return f"{color}{message}{self.RESET}" if color else message

logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)

if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        "%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    ch.setFormatter(formatter)
    logger.addHandler(ch)

### Gestion d'état des notebooks

Implémentation de la classe `NotebookState` :
- Chargement/sauvegarde de notebooks
- Exécution via Papermill
- Mécanisme de cache et validation par hash MD5
- Gestion des états d'approbation

In [39]:
import os
import json
import nbformat
import papermill as pm
import hashlib
import logging


class NotebookState:
    """
    Version simplifiée de la gestion d'un notebook :
      - Charge un notebook depuis `notebook_path`
      - Met à jour directement sur disque lors de `update_cell()`
      - Exécute directement en place (papermill input_path = output_path)
    """

    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
        self._cached_notebook = None
        self._is_approved = False
        self._load_notebook()

    def _load_notebook(self):
        """Lit le notebook sur disque et le stocke en mémoire."""
        if not os.path.exists(self.notebook_path):
            raise FileNotFoundError(f"Notebook introuvable: {self.notebook_path}")

        with open(self.notebook_path, "r", encoding="utf-8") as f:
            self._cached_notebook = nbformat.read(f, as_version=4)

        logger.debug(f"Notebook '{self.notebook_path}' chargé, "
                     f"nb de cellules={len(self._cached_notebook.cells)}.")

    def get_notebook_json(self) -> str:
        """
        Retourne le contenu du notebook en mémoire au format JSON.
        (Ne relance plus Papermill automatiquement).
        """
        raw_json = json.dumps(self._cached_notebook, indent=2, ensure_ascii=False)
        return raw_json

    # Dans la classe NotebookState, ajouter une méthode pour logger le JSON
    def log_notebook_state(self):
        """Log le contenu JSON du notebook (version simplifiée pour les logs)."""
        notebook_json = self.get_notebook_json()
        logger.debug(f"[NotebookState] Current notebook state (simplified):\n{notebook_json[:10000]}...")  # Truncate pour lisibilité


    def update_cell(self, cell_index: int, new_source: str):
        """Modifie immédiatement la source d'une cellule, puis sauvegarde sur disque."""
        old = self._cached_notebook.cells[cell_index].source
        self._cached_notebook.cells[cell_index].source = new_source

        logger.info(f"[NotebookState] Mise à jour de la cellule {cell_index}")
        logger.debug(f"Ancien contenu:\n{old}")
        logger.debug(f"Nouveau contenu:\n{new_source}")

        # Sauvegarde immédiate
        self.save_notebook()
        logger.debug(f"[update_cell] Modification et sauvegarde effectuées pour la cellule {cell_index}.")

    def execute_notebook(self):
        """
        Exécute le notebook "en place", i.e. input=output sur le même fichier.
        On sauvegarde d'abord pour être sûr que le disque est à jour.
        """
        logger.info(f"[NotebookState] Exécution Papermill sur {self.notebook_path} (in place).")
        self.save_notebook()  # S'assure que la version disque est à jour avant exécution

        try:
            with open('execution.log', 'w') as log_file:
                pm.execute_notebook(
                    self.notebook_path,
                    self.notebook_path,
                    kernel_name="python3",
                    progress_bar=False,
                    log_output=True,
                    stdout_file=log_file,
                    stderr_file=log_file
                )
                
             # Analyse des erreurs
            with open('execution.log', 'r') as f:
                logs = f.read()
                if "Error" in logs or "Exception" in logs:
                    logger.error("Erreurs détectées pendant l'exécution :")
                    logger.error(logs[-2000:])  # Affiche les derniers logs

        except Exception as e:
            logger.error(f"[execute_notebook] Erreur lors de l'exécution: {e}")
            raise
        finally:
            self._load_notebook()
        logger.info("[NotebookState] Exécution OK, notebook mis à jour (in place).")

    def save_notebook(self, path: str = ""):
        """
        Sauvegarde en JSON. 
        Si `path` est vide, on réécrit dans `self.notebook_path`.
        """
        if not path:
            path = self.notebook_path

        # On calcule un hash avant sauvegarde (optionnel)
        current_content = json.dumps(self._cached_notebook, sort_keys=True)
        current_hash = hashlib.md5(current_content.encode('utf-8')).hexdigest()

        try:
            with open(path, "w", encoding="utf-8") as f:
                nbformat.write(self._cached_notebook, f)
            logger.info(f"[NotebookState] Notebook sauvegardé sous {path}")
        except Exception as e:
            logger.error(f"[save_notebook] Erreur de sauvegarde: {e}")
            raise

        # Vérification après sauvegarde (optionnel)
        try:
            with open(path, "r", encoding="utf-8") as f:
                saved_notebook = nbformat.read(f, as_version=4)
            saved_content = json.dumps(saved_notebook, sort_keys=True)
            saved_hash = hashlib.md5(saved_content.encode('utf-8')).hexdigest()
        except Exception as e:
            logger.error(f"[save_notebook] Erreur lors de la relecture du notebook: {e}")
            raise

        if current_hash != saved_hash:
            logger.warning("[save_notebook] Le contenu relu ne correspond pas au contenu en mémoire!")
        else:
            logger.debug("[save_notebook] Vérification OK, contenu identique.")

        # On met à jour en mémoire si on a sauvegardé dans un autre path
        if path != self.notebook_path:
            self.notebook_path = path
            self._load_notebook()

    # Optionnel : gestion de l'approbation
    def is_approved(self) -> bool:
        return self._is_approved

    def set_approved(self, approved: bool = True):
        self._is_approved = approved
        logger.info(f"[NotebookState] set_approved({approved}).")


### Validation du système d'état

Procédure de test complète :
1. Copie d'un template de notebook
2. Injection dynamique d'une description de tâche
3. Exécution automatique du notebook
4. Vérification de l'intégrité du fichier

In [40]:
import shutil
import logging



# Exemple de texte de tâche DBpedia
DBPEDIA_TASK_DESCRIPTION = (
    "Créer un notebook Python permettant de requêter DBpedia (SPARQL) et d’afficher "
    "un graphique final avec rdflib ou SPARQLWrapper et plotly.\n\n"
    "**Résultat attendu** :\n"
    "1. Une requête complexe sur DBpedia (avec agrégats) correctement exécutée.\n"
    "2. Un graphique plotly **pertinent** reflétant les données issues de DBpedia.\n"
)

def apply_task_description(notebook_state: NotebookState, task_description: str) -> bool:
    """Injecte la description de tâche dans la cellule appropriée."""
    updated = False
    for idx, cell in enumerate(notebook_state._cached_notebook.cells):
        if cell.cell_type == "markdown" and "## 0. Objectif du Notebook" in cell.source:
            new_source = cell.source
            if "{{TASK_DESCRIPTION}}" in new_source:
                new_source = new_source.replace(
                    "{{TASK_DESCRIPTION}}",
                    f"```markdown\n{task_description}\n```"  # Bloc markdown pour la lisibilité
                )
                
                # La méthode update_cell() de NotebookState
                # sauvegarde immédiatement le fichier modifié.
                notebook_state.update_cell(idx, new_source)
                
                logger.debug(f"[apply_task_description] Cellule {idx} mise à jour.")
                updated = True
            # On quitte la boucle une fois la bonne cellule trouvée.
            break
    return updated


SOURCE_NOTEBOOK = "Notebook-Template.ipynb"
DEST_NOTEBOOK = "Notebook-TaskDBpedia.ipynb"

# 1) On copie d'abord le template pour ne pas le toucher.
shutil.copy2(SOURCE_NOTEBOOK, DEST_NOTEBOOK)
logger.info(f"Copie du template -> {DEST_NOTEBOOK}")

# 2) On charge le nouveau notebook dans NotebookState
notebook_state = NotebookState(DEST_NOTEBOOK)

# 3) On injecte la description DBpedia
changed = apply_task_description(notebook_state, DBPEDIA_TASK_DESCRIPTION)

if changed:
    logger.info("Le placeholder DBpedia a bien été injecté.")

    # 4) Exécution du notebook en place (optionnel)
    logger.info("Exécution du notebook en place...")
    notebook_state.execute_notebook()  
    logger.info("Notebook ré-exécuté après injection de la tâche DBpedia.")

else:
    logger.warning("Aucun placeholder n'a été trouvé. Vérifiez que {{TASK_DESCRIPTION}} est présent.")


2025-02-11 23:25:20,046 [INFO] Orchestration - Copie du template -> Notebook-TaskDBpedia.ipynb
2025-02-11 23:25:20,053 [DEBUG] Orchestration - Notebook 'Notebook-TaskDBpedia.ipynb' chargé, nb de cellules=11.
2025-02-11 23:25:20,055 [INFO] Orchestration - [NotebookState] Mise à jour de la cellule 1
2025-02-11 23:25:20,056 [DEBUG] Orchestration - Ancien contenu:
## 0. Objectif du Notebook

Dans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.

### Tâche originale

Voilà la tâche telle qu'elle a été initialement formulée :

{{TASK_DESCRIPTION}}


### Interprétation et sous-objectifs

Décrire ici l'interprétation de la tâche et les étapes prévues pour la réaliser.
2025-02-11 23:25:20,058 [DEBUG] Orchestration - Nouveau contenu:
## 0. Objectif du Notebook

Dans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.

### Tâche originale

Voilà la tâche telle qu'elle a é

2025-02-11 23:25:22,903 [DEBUG] Orchestration - Notebook 'Notebook-TaskDBpedia.ipynb' chargé, nb de cellules=11.
2025-02-11 23:25:22,905 [INFO] Orchestration - [NotebookState] Exécution OK, notebook mis à jour (in place).
2025-02-11 23:25:22,906 [INFO] Orchestration - Notebook ré-exécuté après injection de la tâche DBpedia.


### Architecture de plugins

Système modulaire pour interagir avec le notebook :
- `BaseNotebookPlugin` : fonctionnalités de base
- `CoderPlugin` : édition des cellules
- `ReviewerPlugin` : validation par exécution
- `AdminPlugin` : approbation finale

In [41]:
from semantic_kernel.functions import kernel_function

class BaseNotebookPlugin:
    """Classe de base pour manipuler NotebookState."""
    def __init__(self, state: NotebookState):
        self.state = state

    @kernel_function(
        name="get_notebook_content",
        description="Renvoie le notebook complet au format JSON."
    )
    def get_notebook_content(self) -> str:
        logger.debug("[Plugin] get_notebook_content appelé.")
        return self.state.get_notebook_json()


class CoderNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="update_cell",
        description="Met à jour la source d'une cellule (par son index)."
    )
    def update_cell(self, cell_index_str: str, new_source: str) -> str:
        logger.info(f"[CoderNotebookPlugin] update_cell({cell_index_str}) appelé.")
        try:
            cell_index = int(cell_index_str)
            self.state.update_cell(cell_index, new_source)
            return f"Cellule {cell_index} mise à jour avec succès."
        except Exception as e:
            logger.error(f"[CoderNotebookPlugin] Erreur lors de update_cell: {str(e)}")
            return f"Erreur lors de la mise à jour: {str(e)}"


class ReviewerNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="validate_notebook",
        description="Exécute le notebook via papermill, renvoie un résumé."
    )
    def validate_notebook(self) -> str:
        logger.info("[ReviewerNotebookPlugin] validate_notebook() appelé => exécution.")
        self.state.execute_notebook()
        return "Notebook exécuté et validé (potentiellement)."


class AdminNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="approve_notebook",
        description="Marque le notebook comme approuvé."
    )
    def approve_notebook(self) -> str:
        logger.debug("[AdminNotebookPlugin] approve_notebook() appelé.")
        self.state.set_approved(True)
        logger.info("[AdminNotebookPlugin] Notebook APPROVED.")
        return "Notebook APPROVED."


### Stratégies d'orchestration

Politiques de contrôle du flux :
- `ApprovedBasedTermination` : arrêt sur approbation ou limite d'étapes
- `NotebookAwareSelection` : rotation cyclique entre agents
- Intégration des états du notebook dans les décisions

In [42]:
from pydantic import PrivateAttr

class ApprovedBasedTerminationStrategy(TerminationStrategy):
    _state: NotebookState = PrivateAttr()
    _max_steps: int = PrivateAttr(default=10)  # Limite de sécurité

    def __init__(self, state: NotebookState, max_steps: int = 10):
        super().__init__()
        self._state = state
        self._max_steps = max_steps
        self._current_step = 0

    async def should_agent_terminate(self, agent, history):
        self._current_step += 1
        is_approved = self._state.is_approved()
        logger.debug(
            f"[TerminationStrategy] Step={self._current_step}/{self._max_steps}, "
            f"IsApproved={is_approved}"
        )

        if is_approved:
            logger.info("[TerminationStrategy] Notebook approuvé => on arrête.")
            return True

        if self._current_step >= self._max_steps:
            logger.warning(f"[TerminationStrategy] max_steps={self._max_steps} atteint => on arrête.")
            return True

        return False



class NotebookAwareSelectionStrategy(SelectionStrategy):
    _state: NotebookState = PrivateAttr()
    _index: int = PrivateAttr(default=-1)

    def __init__(self, state: NotebookState):
        super().__init__()
        self._state = state
        self._index = -1

    def reset(self) -> None:
        self._index = -1

    async def select_agent(self, agents, history):
        logger.debug(f"[SelectionStrategy] Nombre d'agents: {len(agents)}")
        
        if self._state.is_approved():
            logger.info("[SelectionStrategy] Le notebook est déjà approuvé.")
            return None
        
        if not agents:
            logger.warning("[SelectionStrategy] Auncun agent disponible.")
            return None
        
        self._index = (self._index + 1) % len(agents)
        selected = agents[self._index]
        logger.debug(f"[SelectionStrategy] L'agent sélectionné est: {selected.name}")
        
        return selected

### Création des agents intelligents

Initialisation des trois agents spécialisés :
1. **CoderAgent** : 
   - Capacité d'édition via fonctions natives
   - Accès complet au contenu du notebook
   - Instructions détaillées pour les modifications

2. **ReviewerAgent** :
   - Exécution systématique après modifications
   - Analyse des résultats et détection d'erreurs
   - Feedback structuré pour le Coder

3. **AdminAgent** :
   - Décision finale d'approbation
   - Supervision du processus global
   - Verrouillage du workflow

In [43]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments

def create_kernel_for_agent(agent_id: str, plugin_instance) -> Kernel:
    k = Kernel()
    
    # 1) Ajout du service ChatCompletion
    k.add_service(OpenAIChatCompletion(service_id="default"))
    
    # 2) Ajout du plugin
    k.add_plugin(plugin_instance, plugin_name="notebook_plugin")
    return k

# Instanciation des plugins
coder_plugin = CoderNotebookPlugin(notebook_state)
reviewer_plugin = ReviewerNotebookPlugin(notebook_state)
admin_plugin = AdminNotebookPlugin(notebook_state)

# Kernels
coder_kernel = create_kernel_for_agent("coder_kernel", coder_plugin)
reviewer_kernel = create_kernel_for_agent("reviewer_kernel", reviewer_plugin)
admin_kernel = create_kernel_for_agent("admin_kernel", admin_plugin)

# Configuration auto-invocation des fonctions
coder_settings = coder_kernel.get_prompt_execution_settings_from_service_id("default")
coder_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

reviewer_settings = reviewer_kernel.get_prompt_execution_settings_from_service_id("default")
reviewer_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

admin_settings = admin_kernel.get_prompt_execution_settings_from_service_id("default")
admin_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Agents
coder_agent = ChatCompletionAgent(
    service_id="coder_kernel",
    kernel=coder_kernel,
    name="CoderAgent",
    instructions=(
        "Vous êtes le **Coder**. Votre rôle :\n"
        "1. Examiner le contenu actuel du notebook avec 'get_notebook_content'.\n"
        "2. Appliquer des modifications **au besoin** pour atteindre l'objectif global (ou suite aux retours du Reviewer).\n"
        "3. Pour modifier une cellule, utilisez 'update_cell(cell_index, new_source)'.\n"
        "4. Les index de cellules commencent à 0.\n"
        "5. Évitez d'ajouter de nouvelles cellules, sauf indication explicite.\n"
        "6. Quand vous avez fini, résumez vos changements.\n"
    ),
    arguments=KernelArguments(settings=coder_settings)
)

reviewer_agent = ChatCompletionAgent(
    service_id="reviewer_kernel",
    kernel=reviewer_kernel,
    name="ReviewerAgent",
    instructions=(
        "Vous êtes le **Reviewer**. \n"
        "1) Exécutez le notebook pour vérifier qu'il fonctionne (function 'validate_notebook').\n"
        "2) S'il y a une erreur ou un point à améliorer pour l'objectif annoncé, expliquez clairement ce qui ne va pas \n"
        "   et sollicitez CoderAgent pour corriger.\n"
        "3) Si tout est bon, annoncez simplement « OK pour moi ».\n"
    ),
    arguments=KernelArguments(settings=reviewer_settings)
)

admin_agent = ChatCompletionAgent(
    service_id="admin_kernel",
    kernel=admin_kernel,
    name="AdminAgent",
    instructions=(
        "Vous êtes l'**Admin**. \n"
        "1) Ne validez définitivement le notebook que si le ReviewerAgent dit « OK ». \n"
        "2) Pour approuver, appelez 'approve_notebook()'. \n"
        "3) Sinon, redirigez vers CoderAgent pour d'éventuelles modifications.\n"
    ),
    arguments=KernelArguments(settings=admin_settings)
)


# GroupChat avec stratégies
termination_strategy = ApprovedBasedTerminationStrategy(notebook_state)
selection_strategy = NotebookAwareSelectionStrategy(notebook_state)

group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    selection_strategy=selection_strategy,
    termination_strategy=termination_strategy
)

logger.info("Agents créés, group_chat initialisé.")


2025-02-11 23:25:24,279 [INFO] Orchestration - Agents créés, group_chat initialisé.


### Boucle de conversation principale

Mécanisme d'exécution :
- Historique de conversation persistant
- Logging détaillé des appels de fonction
- Gestion asynchrone des interactions
- Conditions de sortie multiples
- Archivage des versions intermédiaires

In [44]:
import asyncio
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent

async def run_conversation():
    try:
        logger.info("Version initiale du  notebook")
        notebook_state.log_notebook_state()
        initial_content = notebook_state.get_notebook_json()
        group_chat.history.add_system_message(f"NOTEBOOK CONTENT:\\n{initial_content}")
        group_chat.history.add_user_message(
            "Bonjour, j'aimerais finaliser ce notebook qui contient des instructions et des cellules à compléter..."
        )
        
        iteration = 0
        async for message in group_chat.invoke():
            iteration += 1
            logger.debug(f"--- Iteration #{iteration} ---")
            
            # Log des appels de fonction
            for item in message.items:
                if isinstance(item, FunctionCallContent):
                    logger.info(f"FUNCTION CALL: {item.name}({item.arguments})")
                if isinstance(item, FunctionResultContent):
                    logger.info(f"FUNCTION RESULT: {item.result}")

            role = message.role.name
            content = message.content
            logger.info(f"{role.upper()} >>> {content}")

            if notebook_state.is_approved():
                logger.info("Notebook approuvé - arrêt de la conversation")
                break
            
        
        logger.info("Version finale sauvegardée dans le notebook")
        notebook_state.log_notebook_state()

    except Exception as e:
        logger.error(f"Erreur inattendue: {str(e)}")
    finally:
        logger.info(f"Statut final - Approuvé: {notebook_state.is_approved()}")
        logger.info("Conversation terminée.")
        
        
await run_conversation()


2025-02-11 23:25:24,304 [INFO] Orchestration - Version initiale du  notebook
2025-02-11 23:25:24,309 [DEBUG] Orchestration - [NotebookState] Current notebook state (simplified):
{
  "cells": [
    {
      "cell_type": "markdown",
      "id": "516d2854",
      "metadata": {
        "papermill": {
          "duration": 0.002846,
          "end_time": "2025-02-11T22:25:22.417934",
          "exception": false,
          "start_time": "2025-02-11T22:25:22.415088",
          "status": "completed"
        },
        "tags": []
      },
      "source": "# Notebook de travail\n\nCe notebook est généré de façon incrémentale pour accomplir la tâche décrite ci-dessous."
    },
    {
      "cell_type": "markdown",
      "id": "2a226db6",
      "metadata": {
        "papermill": {
          "duration": 0.002989,
          "end_time": "2025-02-11T22:25:22.424933",
          "exception": false,
          "start_time": "2025-02-11T22:25:22.421944",
          "status": "completed"
        },
        "t